# Data Collection and Aggregation

## Half-life

In [ ]:
## Imports

import pandas as pd
import numpy as np
import re

import pubchempy as pcp

In [ ]:
## Functions

def standardizeDF(dataframe, ref, endpoint):
    df = dataframe.copy()
    # Get endpoint value
    df['value'] = df[endpoint]
    # Add reference
    df['ref'] = ref
    # Add endpoint name
    df['endpoint'] = endpoint

    # Return the final dataframe
    return df[['smiles','value','ref','endpoint']]

### Fan et al.

In [ ]:
# Load data
FanEtAl_df = pd.read_excel('../raw_data/half_life/FanEtAl_dataset.xlsx')
FanEtAl_df = FanEtAl_df[['smiles','t12']] # select columns

# "Clean" SMILES using a regular expression
# def clean_smiles(smiles):
#     return re.sub(r"\[([n,N,S])\+[\@]*([H]*)([1-3]*)]", r"[\1\2\3+]", smiles)
# FanEtAl_df['SMILES'] = FanEtAl_df['smiles'].apply(clean_smiles)

# Convert half-life values from hours to the logarithmic scale (base 10) 
FanEtAl_df['logHL'] = np.log10(FanEtAl_df['t12'])

# Generate the standardized dataset
FanEtAl_std_df = standardizeDF(FanEtAl_df, ref='Fan', endpoint='logHL')
FanEtAl_std_df

### Lombardo et al.

In [ ]:
# Load data
LombardoEtAl_df = pd.read_excel('../raw_data/half_life/LombardoEtAl_dataset.xls', skiprows=8)
LombardoEtAl_df = LombardoEtAl_df[['SMILES','terminal  t1/2 (h)']] # select columns
LombardoEtAl_df = LombardoEtAl_df.rename(columns={'SMILES':'smiles'})

# Convert half-life values from hours to the logarithmic scale (base 10) 
LombardoEtAl_df['logHL'] = np.log10(LombardoEtAl_df['terminal  t1/2 (h)'])

# Generate the standardized dataset
LombardoEtAl_std_df = standardizeDF(LombardoEtAl_df, ref='Lombardo', endpoint='logHL')
LombardoEtAl_std_df

### DDPD 1.0: Half-life

In [ ]:
# Load data
DDPD_df = pd.read_csv('../raw_data/half_life/DDPD.tsv', sep='\t')
# Select half-life data
DDPD_df = DDPD_df.loc[DDPD_df['prop'] == 'Half-life']
# Discard molecules without annotated half-life unit
DDPD_df = DDPD_df.loc[DDPD_df['unit'].notna()]
DDPD_df = DDPD_df[['smiles','value']] # select columns

# Convert half-life values from hours to the logarithmic scale (base 10) 
DDPD_df['logHL'] = np.log10(DDPD_df['value'])

# Generate the standardized dataset
DDPD_std_df = standardizeDF(DDPD_df, ref='DDPD', endpoint='logHL')
DDPD_std_df

### e-Drug 3D: Half-life

In [ ]:
# Load data
eDrug3D_df = pd.read_csv('../raw_data/half_life/e-Drug3D.txt', sep=';')
eDrug3D_df.columns = eDrug3D_df.columns.str.strip() # strip spaces from column names
eDrug3D_df = eDrug3D_df.applymap(lambda x: x.strip() if isinstance(x, str) else x) # strip spaces from all cell values
# Select drugs with annotated half-life value
eDrug3D_df = eDrug3D_df.loc[eDrug3D_df['t1/2(hour)'] != '']
eDrug3D_df = eDrug3D_df.loc[~eDrug3D_df['t1/2(hour)'].isin(['SEVERAL'])] # exclude the drug with 'SEVERAL' annotated value
eDrug3D_df = eDrug3D_df[['Name','CAS','t1/2(hour)']] # select columns

# Retrieve SMILES using PubChemPy
eDrug3D_df['smiles'] = eDrug3D_df['CAS'].apply(lambda cas: {cmp.canonical_smiles for cmp in pcp.get_compounds(cas, 'name')})
# eDrug3D_df['n_smiles'] = eDrug3D_df['smiles'].apply(len)
# Fix conflicting molecules
eDrug3D_df.loc[179, 'smiles'] = {'COC(=O)C(C1CCCCN1)C2=CC=CC=C2'}
eDrug3D_df.loc[193, 'smiles'] = {'CC1(CC(=O)N(C1=O)C)C2=CC=CC=C2'}
eDrug3D_df.loc[259, 'smiles'] = {'CC1CC2C(CCC3(C2CCC3(C(=O)C)OC(=O)C)C)C4(C1=CC(=O)CC4)C'}
eDrug3D_df.loc[337, 'smiles'] = {'C[N+]1=CC=CC=C1C=NO'}
eDrug3D_df.loc[404, 'smiles'] = {'CC(=O)OC1CC2CCC3C(C2(CC1[N+]4(CCCCC4)C)C)CCC5(C3CC(C5OC(=O)C)[N+]6(CCCCC6)C)C'}
eDrug3D_df.loc[453, 'smiles'] = {''} # not conclusive
eDrug3D_df.loc[565, 'smiles'] = {'CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC2=CC=C(C=C2)O)NC(=O)C(CO)NC(=O)C(CC3=CNC4=CC=CC=C43)NC(=O)C(CC5=CN=CN5)NC(=O)C6CCC(=O)N6'}
eDrug3D_df.loc[738, 'smiles'] = {'CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OC3C(OC(C(C3O)O)O)CO)CO)O)O)NC4C=C(C(C(C4O)O)O)CO'}
eDrug3D_df.loc[748, 'smiles'] = {'CC(C)(C1=CC=C(C=C1)C(CCCN2CCC(CC2)C(C3=CC=CC=C3)(C4=CC=CC=C4)O)O)C(=O)O'}
eDrug3D_df.loc[764, 'smiles'] = {'CC(C)(C1=CC=C(C=C1)C(CCCN2CCC(CC2)C(C3=CC=CC=C3)(C4=CC=CC=C4)O)O)C(=O)O'}
eDrug3D_df.loc[867, 'smiles'] = {'CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC'}
eDrug3D_df.loc[976, 'smiles'] = {'CCC(C)CCCCC(=O)NC(CCNCS(=O)(=O)O)C(=O)NC(C(C)O)C(=O)NC(CCNCS(=O)(=O)O)C(=O)NC1CCNC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC1=O)CCNCS(=O)(=O)O)CC(C)C)CC(C)C)CCNCS(=O)(=O)O)CCNCS(=O)(=O)O)C(C)O'} # uncharged molecule
eDrug3D_df.loc[982, 'smiles'] = {'CN(C(=O)NC1C(C(C(OC1O)CO)O)O)N=O'}
eDrug3D_df.loc[1022, 'smiles'] = {'CCNC1CC(C(C(C1OC2C(C(C(CO2)(C)O)NC)O)O)OC3C(CC=C(O3)CN)N)N'}
eDrug3D_df.loc[1036, 'smiles'] = {'CN(C(=O)NC1C(C(C(OC1O)CO)O)O)N=O'}
eDrug3D_df.loc[1060, 'smiles'] = {'CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=NC6(CCN(CC6)CC(C)C)N=C25)O)C'}
eDrug3D_df.loc[1212, 'smiles'] = {'CCC(C)C1C(=O)N2CCCC2C3(N1C(=O)C(O3)(C(C)C)NC(=O)C4CC5C(CC6=CNC7=CC=CC5=C67)N(C4)C)O.CC(C)CC1C(=O)N2CCCC2C3(N1C(=O)C(O3)(C(C)C)NC(=O)C4CC5C(CC6=CNC7=CC=CC5=C67)N(C4)C)O.CC(C)C1C(=O)N2CCCC2C3(N1C(=O)C(O3)(C(C)C)NC(=O)C4CC5C(CC6=CNC7=CC=CC5=C67)N(C4)C)O.CC(C)C1(C(=O)N2C(C(=O)N3CCCC3C2(O1)O)CC4=CC=CC=C4)NC(=O)C5CC6C(CC7=CNC8=CC=CC6=C78)N(C5)C'}
eDrug3D_df.loc[1233, 'smiles'] = {'C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N'}
eDrug3D_df.loc[1426, 'smiles'] = {'COC(=O)C(C1CCCCN1)C2=CC=CC=C2'}
eDrug3D_df.loc[1474, 'smiles'] = {'C1CC1C2=NC3=CC=CC=C3C(=C2C=CC(CC(CC(=O)O)O)O)C4=CC=C(C=C4)F'}
eDrug3D_df.loc[1536, 'smiles'] = {'C1CC1COC2=C(C=CC(=C2)C(=O)NC3=C(C=NC=C3Cl)Cl)OC(F)F'} # ROFLUMILAST
eDrug3D_df.loc[1561, 'smiles'] = {'CCC(=O)OC1C(CC2C1(CCC3C2CCC4C3(CC(C(C4)OC(=O)C)N5CCCCC5)C)C)[N+]6(CCCCC6)CC=C'} # RAPACURONIUM
eDrug3D_df.loc[1576, 'smiles'] = {'C1C(CN2C1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN'}
eDrug3D_df.loc[1606, 'smiles'] = {'C1=CC=C(C=C1)CC(=O)O'} # uncharged molecule 
eDrug3D_df.loc[1631, 'smiles'] = {'CCCCCCCCCCCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO'}
eDrug3D_df.loc[1639, 'smiles'] = {'C=C1C(CC(C1CO)O)N2C=NC3=C2N=C(NC3=O)N'} # ENTECAVIR
eDrug3D_df.loc[1675, 'smiles'] = {'CN(C)CC1CCCCC1(C2=CC(=CC=C2)OC)O'} # TRAMADOL
eDrug3D_df.loc[1937, 'smiles'] = {'CC(C)OC1=C(C=C(C=C1)C2=NC(=NO2)C3=C4CCC(C4=CC=C3)NCCO)C#N'} # OZANIMOD
eDrug3D_df.loc[1990, 'smiles'] = {'CCN1C2=CC(=NC=C2C=C(C1=O)C3=CC(=C(C=C3Br)F)NC(=O)NC4=CC=CC=C4)NC'} # RIPRETINIB
eDrug3D_df.loc[2035, 'smiles'] = {'CN1C(=CN=C1COC2=CC=C(C=C2)SC)[N+](=O)[O-]'} # FEXINIDAZOLE
eDrug3D_df.loc[2046, 'smiles'] = {'CC(C)OC(=O)C1=CN=C(N=C1C2=CN(C3=CC=CC=C32)C)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC'} # MOBOCERTINIB
eDrug3D_df.loc[2047, 'smiles'] = {'CC(C)OC(=O)C1=CN=C(N=C1C2=CN(C3=CC=CC=C32)C)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC'} # MOBOCERTINIB
eDrug3D_df.loc[2098, 'smiles'] = {'CC(C)(C)C1=CC(=NO1)NC(=O)NC2=CC=C(C=C2)C3=CN4C5=C(C=C(C=C5)OCCN6CCOCC6)SC4=N3'} # QUIZARTINIB
eDrug3D_df.loc[2117, 'smiles'] = {'CC1=CC(=CC(=C1OC(C)(C)C(=O)O)C)C=CC(=O)C2=CC=C(C=C2)SC'} # ELAFIBRANOR
# Get the final dataset
eDrug3D_df = eDrug3D_df.explode('smiles')

# Convert half-life values from hours to the logarithmic scale (base 10) 
eDrug3D_df['t1/2(hour)'] = eDrug3D_df['t1/2(hour)'].astype(float) # string to float conversion
eDrug3D_df['logHL'] = np.log10(eDrug3D_df['t1/2(hour)'])

# Generate the standardized dataset
eDrug3D_std_df = standardizeDF(eDrug3D_df, ref='eDrug3D', endpoint='logHL')
eDrug3D_std_df

### Obach et al. (TDC benchmark)

NOTE: Although the  Obach et al.'s dataset corresponds to the Therapeutic Data Commons (TDC) benchmarking dataset for the half-life endpoint, the data was retrieved from the original source.

In [ ]:
# Load data
ObachEtAl_df = pd.read_excel('../raw_data/half_life/ObachEtAl.xls', skiprows=1)
ObachEtAl_df = ObachEtAl_df[['Name','CAS #','t1/2 (h)']] # select columns

# Retrieve SMILES using PubChemPy
ObachEtAl_df['smiles'] = ObachEtAl_df['CAS #'].apply(lambda cas: {cmp.canonical_smiles for cmp in pcp.get_compounds(cas, 'name')})
# ObachEtAl_df['n_smiles'] = ObachEtAl_df['smiles'].apply(len)
# Fix conflicting molecules
ObachEtAl_df.loc[2, 'smiles'] = {'CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OC3C(OC(C(C3O)O)O)CO)CO)O)O)NC4C=C(C(C(C4O)O)O)CO'}
ObachEtAl_df.loc[18, 'smiles'] = {'COC1=CC2=C(C=C1C(=O)NCC3CCCN3CC=C)NN=N2'}
ObachEtAl_df.loc[178, 'smiles'] = {'C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N'}
ObachEtAl_df.loc[278, 'smiles'] = {'COCCNC(=O)CN(CCN(CCN(CC(=O)NCCOC)CC(=O)O)CC(=O)O)CC(=O)O.[Gd]'} # uncharged molecule
ObachEtAl_df.loc[355, 'smiles'] = {'CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC2=CC=C(C=C2)O)NC(=O)C(CO)NC(=O)C(CC3=CNC4=CC=CC=C43)NC(=O)C(CC5=CN=CN5)NC(=O)C6CCC(=O)N6'}
ObachEtAl_df.loc[438, 'smiles'] = {'CCNC1CC(C(C(C1OC2C(C(C(CO2)(C)O)NC)O)O)OC3C(CC=C(O3)CN)N)N'}
ObachEtAl_df.loc[468, 'smiles'] = {'CC(=O)OC1CC2CCC3C(C2(CC1[N+]4(CCCCC4)C)C)CCC5(C3CC(C5OC(=O)C)[N+]6(CCCCC6)C)C'}
ObachEtAl_df.loc[533, 'smiles'] = {'CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=NC6(CCN(CC6)CC(C)C)N=C25)O)C'}
ObachEtAl_df.loc[579, 'smiles'] = {'CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC'}
ObachEtAl_df.loc[609, 'smiles'] = {'CCCCS(=O)(=O)NC(CC1=CC=C(C=C1)OCCCCC2CCNCC2)C(=O)O'}
ObachEtAl_df.loc[638, 'smiles'] = {'CN1CCC2=CC(=C3C=C2C1CC4=CC=C(C=C4)OC5=C6C(CC7=CC(=C(C=C7)O)O3)[N+](CCC6=CC(=C5O)OC)(C)C)OC'}
# Get the final dataset
ObachEtAl_df = ObachEtAl_df.explode('smiles')

# Convert half-life values from hours to the logarithmic scale (base 10) 
ObachEtAl_df['logHL'] = np.log10(ObachEtAl_df['t1/2 (h)'])

# Generate the standardized dataset
ObachEtAl_std_df = standardizeDF(ObachEtAl_df, ref='Obach', endpoint='logHL')
ObachEtAl_std_df

### Dataset Aggregation

In [ ]:
HL_dataset = pd.concat([FanEtAl_std_df, LombardoEtAl_std_df, DDPD_std_df, eDrug3D_std_df, ObachEtAl_std_df], axis=0, ignore_index=True)
HL_dataset.to_csv('../data/half_life/logHL_aggregated_dataset.tsv', sep='\t', index=False)

## Clearance

### Lombardo et al.

In [ ]:
# Load data
LombardoEtAl_df = pd.read_excel('../raw_data/clearance/LombardoEtAl_dataset.xls', skiprows=8)
LombardoEtAl_df = LombardoEtAl_df[['SMILES','human CL (mL/min/kg)']] # select columns
LombardoEtAl_df = LombardoEtAl_df.rename(columns={'SMILES':'smiles'})

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
LombardoEtAl_df['logCL'] = np.log10(LombardoEtAl_df['human CL (mL/min/kg)'])

# Generate the standardized dataset
LombardoEtAl_std_df = standardizeDF(LombardoEtAl_df, ref='Lombardo', endpoint='logCL')
LombardoEtAl_std_df

### Astrazeneca (TDC benchmark, original source)

NOTE: Although the Astrazeneca dataset corresponds to the Therapeutic Data Commons (TDC) benchmarking dataset for the clearance endpoint, the data was retrieved from the original source.

In [ ]:
# Load data
Astrazeneca_df = pd.read_csv('../raw_data/clearance/Astrazeneca_dataset.tsv', sep='\t')
# Select human microsome clearance data
Astrazeneca_microsome_df = Astrazeneca_df.loc[(Astrazeneca_df['Standard Type'] == 'CL') & (Astrazeneca_df['Standard Units'] == 'mL.min-1.g-1') & (Astrazeneca_df['Assay Organism'] == 'Homo sapiens')]
# Exclude referential data
Astrazeneca_microsome_df = Astrazeneca_microsome_df.loc[Astrazeneca_microsome_df['Standard Relation'].isna()]
Astrazeneca_microsome_df = Astrazeneca_microsome_df[['Smiles','Standard Value']] # select columns
Astrazeneca_microsome_df = Astrazeneca_microsome_df.rename(columns={'Smiles':'smiles'}) # rename columns

# Convert uL/min/mg to ml/min/kg (data scaling)
Astrazeneca_microsome_df['CL'] = Astrazeneca_microsome_df['Standard Value'] * 0.945
# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
Astrazeneca_microsome_df['logCL'] = np.log10(Astrazeneca_microsome_df['CL'])

# Select human hepatocyte clearance data
Astrazeneca_hepatocyte_df = Astrazeneca_df.loc[(Astrazeneca_df['Standard Type'] == 'CL') & (Astrazeneca_df['Standard Units'] == 'uL.min-1.(10^6cells)-1') & (Astrazeneca_df['Assay Organism'] == 'Homo sapiens')]
# Exclude referential data
Astrazeneca_hepatocyte_df = Astrazeneca_hepatocyte_df.loc[Astrazeneca_hepatocyte_df['Standard Relation'].isna()]
Astrazeneca_hepatocyte_df = Astrazeneca_hepatocyte_df[['Smiles','Standard Value']] # select columns
Astrazeneca_hepatocyte_df = Astrazeneca_hepatocyte_df.rename(columns={'Smiles':'smiles'}) # rename columns

# Convert uL/min/MCells to ml/min/kg (data scaling)
Astrazeneca_hepatocyte_df['CL'] = Astrazeneca_hepatocyte_df['Standard Value'] * 2.52
# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
Astrazeneca_hepatocyte_df['logCL'] = np.log10(Astrazeneca_hepatocyte_df['CL'])

# Aggregate both Astrazeneca intrinsic clearance datasets into a single one
Astrazeneca_aggregated_df = pd.concat([Astrazeneca_microsome_df, Astrazeneca_hepatocyte_df], axis=0, ignore_index=True)
# Generate the standardized dataset
Astrazeneca_std_df = standardizeDF(Astrazeneca_aggregated_df, ref='Astrazeneca', endpoint='logCL')
Astrazeneca_std_df

### Iwata et al.

In [ ]:
# Load data
IwataEtAl_df = pd.read_excel('../raw_data/clearance/IwataEtAl_dataset.xlsx', sheet_name='Rawdata_Before_Imputation')
IwataEtAl_df = IwataEtAl_df[['mol','human_CL_mL_min_kg']] # select columns
IwataEtAl_df = IwataEtAl_df.rename(columns={'mol':'smiles'}) # rename columns

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
IwataEtAl_df['logCL'] = np.log10(IwataEtAl_df['human_CL_mL_min_kg'])

# Generate the standardized dataset
IwataEtAl_std_df = standardizeDF(IwataEtAl_df, ref='Iwata', endpoint='logCL')
IwataEtAl_std_df

### Obach et al.

In [ ]:
# Load data
ObachEtAl_df = pd.read_excel('../raw_data/clearance/ObachEtAl.xls', skiprows=1)
ObachEtAl_df = ObachEtAl_df[['Name','CAS #','CL (mL/min/kg)']] # select columns

# Retrieve SMILES using PubChemPy
ObachEtAl_df['smiles'] = ObachEtAl_df['CAS #'].apply(lambda cas: {cmp.canonical_smiles for cmp in pcp.get_compounds(cas, 'name')})
# ObachEtAl_df['n_smiles'] = ObachEtAl_df['smiles'].apply(len)
# Fix conflicting molecules
ObachEtAl_df.loc[2, 'smiles'] = {'CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OC3C(OC(C(C3O)O)O)CO)CO)O)O)NC4C=C(C(C(C4O)O)O)CO'}
ObachEtAl_df.loc[18, 'smiles'] = {'COC1=CC2=C(C=C1C(=O)NCC3CCCN3CC=C)NN=N2'}
ObachEtAl_df.loc[178, 'smiles'] = {'C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N'}
ObachEtAl_df.loc[278, 'smiles'] = {'COCCNC(=O)CN(CCN(CCN(CC(=O)NCCOC)CC(=O)O)CC(=O)O)CC(=O)O.[Gd]'} # uncharged molecule
ObachEtAl_df.loc[355, 'smiles'] = {'CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC2=CC=C(C=C2)O)NC(=O)C(CO)NC(=O)C(CC3=CNC4=CC=CC=C43)NC(=O)C(CC5=CN=CN5)NC(=O)C6CCC(=O)N6'}
ObachEtAl_df.loc[438, 'smiles'] = {'CCNC1CC(C(C(C1OC2C(C(C(CO2)(C)O)NC)O)O)OC3C(CC=C(O3)CN)N)N'}
ObachEtAl_df.loc[468, 'smiles'] = {'CC(=O)OC1CC2CCC3C(C2(CC1[N+]4(CCCCC4)C)C)CCC5(C3CC(C5OC(=O)C)[N+]6(CCCCC6)C)C'}
ObachEtAl_df.loc[533, 'smiles'] = {'CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=NC6(CCN(CC6)CC(C)C)N=C25)O)C'}
ObachEtAl_df.loc[579, 'smiles'] = {'CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC'}
ObachEtAl_df.loc[609, 'smiles'] = {'CCCCS(=O)(=O)NC(CC1=CC=C(C=C1)OCCCCC2CCNCC2)C(=O)O'}
ObachEtAl_df.loc[638, 'smiles'] = {'CN1CCC2=CC(=C3C=C2C1CC4=CC=C(C=C4)OC5=C6C(CC7=CC(=C(C=C7)O)O3)[N+](CCC6=CC(=C5O)OC)(C)C)OC'}
# Get the final dataset
ObachEtAl_df = ObachEtAl_df.explode('smiles')

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
ObachEtAl_df['logCL'] = np.log10(ObachEtAl_df['CL (mL/min/kg)'])

# Generate the standardized dataset
ObachEtAl_std_df = standardizeDF(ObachEtAl_df, ref='Obach', endpoint='logCL')
ObachEtAl_std_df

### Gombar and Hall

In [ ]:
# Load data
GombarHall_df = pd.read_excel('../raw_data/clearance/GombarHall_dataset.xlsx')
GombarHall_df = GombarHall_df[['Clean SMILES','CL (ml/min/kg)']] # select columns
GombarHall_df = GombarHall_df.rename(columns={'Clean SMILES':'smiles'}) # rename columns

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
GombarHall_df['logCL'] = np.log10(GombarHall_df['CL (ml/min/kg)'])

# Generate the standardized dataset
GombarHall_std_df = standardizeDF(GombarHall_df, ref='GombarHall', endpoint='logCL')
GombarHall_std_df

### Varma et al. (2009)

In [ ]:
# Load data
VarmaEtAl_2009_df = pd.read_excel('../raw_data/clearance/VarmaEtAl_2009_dataset.xls', skiprows=20).dropna(subset=["Name"])
VarmaEtAl_2009_df = VarmaEtAl_2009_df[['Name','CAS #','CLtotal (mL/min/kg)']] # select columns

# Retrieve SMILES using PubChemPy
VarmaEtAl_2009_df['smiles'] = VarmaEtAl_2009_df['CAS #'].apply(lambda cas: {cmp.canonical_smiles for cmp in pcp.get_compounds(cas, 'name')})
# VarmaEtAl_2009_df['n_smiles'] = VarmaEtAl_2009_df['smiles'].apply(len)
# Fix conflicting molecules
VarmaEtAl_2009_df.loc[238, 'smiles'] = {'CCNC1CC(C(C(C1OC2C(C(C(CO2)(C)O)NC)O)O)OC3C(CC=C(O3)CN)N)N'}
VarmaEtAl_2009_df.loc[298, 'smiles'] = {'CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=NC6(CCN(CC6)CC(C)C)N=C25)O)C'}
VarmaEtAl_2009_df.loc[333, 'smiles'] = {'CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC'}
VarmaEtAl_2009_df.loc[354, 'smiles'] = {'CCCCS(=O)(=O)NC(CC1=CC=C(C=C1)OCCCCC2CCNCC2)C(=O)O'}
VarmaEtAl_2009_df.loc[372, 'smiles'] = {'CN1CCC2=CC(=C3C=C2C1CC4=CC=C(C=C4)OC5=C6C(CC7=CC(=C(C=C7)O)O3)[N+](CCC6=CC(=C5O)OC)(C)C)OC'}
# Get the final dataset
VarmaEtAl_2009_df = VarmaEtAl_2009_df.explode('smiles')

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
VarmaEtAl_2009_df['logCL'] = np.log10(VarmaEtAl_2009_df['CLtotal (mL/min/kg)'])

# Generate the standardized dataset
VarmaEtAl_2009_std_df = standardizeDF(VarmaEtAl_2009_df, ref='Varma2009', endpoint='logCL')
VarmaEtAl_2009_std_df

### Varma et al. (2010)

In [ ]:
# Load data
VarmaEtAl_2010_df = pd.read_excel('../raw_data/clearance/VarmaEtAl_2010_dataset.xls', skiprows=22).dropna(subset=["Name"])
VarmaEtAl_2010_df = VarmaEtAl_2010_df[['Name','CAS','CLt']] # select columns

# Retrieve SMILES using PubChemPy
VarmaEtAl_2010_df['CAS'] = VarmaEtAl_2010_df['CAS'].apply(lambda cas: cas.strftime('%Y-%m-%d') if isinstance(cas, pd.Timestamp) else str(cas)) # convert 'datetime.datetime' objects into strings
VarmaEtAl_2010_df['smiles'] = VarmaEtAl_2010_df['CAS'].apply(lambda cas: {cmp.canonical_smiles for cmp in pcp.get_compounds(cas, 'name')})
# VarmaEtAl_2010_df['n_smiles'] = VarmaEtAl_2010_df['smiles'].apply(len)
# Fix conflicting molecules
VarmaEtAl_2010_df.loc[1, 'smiles'] = {'CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OC3C(OC(C(C3O)O)O)CO)CO)O)O)NC4C=C(C(C(C4O)O)O)CO'}
VarmaEtAl_2010_df.loc[8, 'smiles'] = {'COC1=CC2=C(C=C1C(=O)NCC3CCCN3CC=C)NN=N2'}
VarmaEtAl_2010_df.loc[9, 'smiles'] = {'C1=NNC2=C1C(=O)NC=N2'}
VarmaEtAl_2010_df.loc[20, 'smiles'] = {'CC1=CC(=O)N(N1C)C2=CC=CC=C2'}
VarmaEtAl_2010_df.loc[49, 'smiles'] = {'CN1C(=NC(=O)C(=O)N1)SCC2=C(N3C(C(C3=O)NC(=O)C(=NOC)C4=CSC(=N4)N)SC2)C(=O)O'}
VarmaEtAl_2010_df.loc[65, 'smiles'] = {'CC1=C(N=CN1)CSCCNC(=NC)NC#N'}
VarmaEtAl_2010_df.loc[77, 'smiles'] = {'C1CNP(=O)(OC1)N(CCCl)CCCl'}
VarmaEtAl_2010_df.loc[91, 'smiles'] = {'CC(C)N(CCC(C1=CC=CC=C1)(C2=CC=CC=N2)C(=O)N)C(C)C'}
VarmaEtAl_2010_df.loc[95, 'smiles'] = {'CC1C(C(C(O1)N2C=C(C(=O)NC2=O)F)O)O'}
VarmaEtAl_2010_df.loc[109, 'smiles'] = {'C1CN(CCC12CNC(=O)O2)CCC3=CC=CC=C3'}
VarmaEtAl_2010_df.loc[118, 'smiles'] = {'C1C(N(C2=C(N1)N=C(NC2=O)N)C=O)CNC3=CC=C(C=C3)C(=O)NC(CCC(=O)O)C(=O)O'}
VarmaEtAl_2010_df.loc[125, 'smiles'] = {'CC1CN(CCN1)C2=C(C=C3C(=C2OC)N(C=C(C3=O)C(=O)O)C4CC4)F'}
VarmaEtAl_2010_df.loc[127, 'smiles'] = {'CC(C1CCC(C(O1)OC2C(CC(C(C2O)OC3C(C(C(CO3)(C)O)NC)O)N)N)N)NC'}
VarmaEtAl_2010_df.loc[162, 'smiles'] = {'CCC(C)C(=O)OC1CC(C=C2C1C(C(C=C2)C)CCC3CC(CC(=O)O3)O)C'}
VarmaEtAl_2010_df.loc[174, 'smiles'] = {'CC1=NC=C(N1CCO)[N+](=O)[O-]'}
VarmaEtAl_2010_df.loc[213, 'smiles'] = {'CC1(C(N2C(S1)C(C2=O)NC(=O)COC3=CC=CC=C3)C(=O)O)C'}
VarmaEtAl_2010_df.loc[217, 'smiles'] = {'CC1CCCC(N1CCCC(C2=CC=CC=C2)(C3=CC=CC=N3)O)C'}
VarmaEtAl_2010_df.loc[234, 'smiles'] = {'CCOC1=CC=CC=C1OC(C2CNCCO2)C3=CC=CC=C3'}
VarmaEtAl_2010_df.loc[238, 'smiles'] = {'CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=NC6(CCN(CC6)CC(C)C)N=C25)O)C'}
VarmaEtAl_2010_df.loc[270, 'smiles'] = {'CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC'}
# Get the final dataset
VarmaEtAl_2010_df = VarmaEtAl_2010_df.explode('smiles')

# Convert clearance values from mL/min/kg to the logarithmic scale (base 10) 
VarmaEtAl_2010_df['logCL'] = np.log10(VarmaEtAl_2010_df['CLt'])

# Generate the standardized dataset
VarmaEtAl_2010_std_df = standardizeDF(VarmaEtAl_2010_df, ref='Varma2010', endpoint='logCL')
VarmaEtAl_2010_std_df

### Dataset Aggregation

In [ ]:
CL_dataset = pd.concat([LombardoEtAl_std_df, Astrazeneca_std_df, IwataEtAl_std_df, ObachEtAl_std_df, GombarHall_std_df, VarmaEtAl_2009_std_df, VarmaEtAl_2010_std_df], axis=0, ignore_index=True)
CL_dataset.to_csv('../data/clearance/logCL_aggregated_dataset.tsv', sep='\t', index=False)